<a href="https://colab.research.google.com/github/willdphan/langchain-lex-agent/blob/main/langchain_lex_agent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [20]:
!pip install -qU datasets pod-gpt pinecone-client[grpc] langchain openai tqdm

Set api keys, etc:

In [21]:
OPENAI_API_KEY = "OPENAI_API_KEY"  # platform.openai.com
PINECONE_API_KEY = "PINECONE_API_KEY"  # app.pinecone.io
PINECONE_ENV = "PINECONE_ENV"

First we download a prebuilt Lex Fridman podcast transcriptions dataset from HF:

In [22]:
from datasets import load_dataset

data = load_dataset(
    'jamescalam/lex-transcripts',
    split='train'
)
data

Dataset({
    features: ['video_id', 'channel_id', 'title', 'published', 'transcript', 'source'],
    num_rows: 499
})

Initialize the `indexer` object. pod_gpt is a python package that provides an implementation of the OpenAI GPT. The Indexer class is used to create an indexer, which allows you to index and search text documents using the language model. 

In [23]:
import pod_gpt

indexer = pod_gpt.Indexer(
    openai_api_key=OPENAI_API_KEY,
    pinecone_api_key=PINECONE_API_KEY,
    pinecone_environment=PINECONE_ENV,
    index_name="pod-gpt"
)

The Python loop iterates through the data, formats the published field of each row, creates a VideoRecord object, and indexes the object to a Pinecone index using the pod_gpt.Indexer class. The tqdm.auto library is used to provide a progress bar for the indexing process.

In each iteration, the code formats the published field of the row object as a string in the '%Y%m%d' format using the strftime() method.

When creating a VideoRecord object, the fields are populated with the corresponding metadata for a particular video.

In [24]:
from tqdm.auto import tqdm

for row in data:
    row['published'] = row['published'].strftime('%Y%m%d')
    indexer(pod_gpt.VideoRecord(**row))

The code then checks if an index with the name "pod-gpt" exists by calling the "list_indexes()" function and checking if the index name is in the list of indexes.

Finally, the code creates an index object for the "pod-gpt" index by calling the "Index()" function and passing in the index name as a parameter.

In [25]:
import pinecone

pinecone.init(
    api_key=PINECONE_API_KEY,  # app.pinecone.io
    environment=PINECONE_ENV  # next to API key in console
)

index_name = "pod-gpt"

if index_name not in pinecone.list_indexes():
    raise ValueError(
        f"No '{index_name}' index exists. You must create the index before "
        "running this notebook. Please refer to the walkthrough at "
        "'github.com/pinecone-io/examples'."  # TODO add full link
    )

index = pinecone.Index(index_name)

Initialize the retrieval components (embedding model and vector DB)

The first two import statements are necessary classes from the "langchain" library to create a vector database.

Next we create a Pinecone object

We create an object of the "Pinecone" class with the "index" parameter set to the value of the "index" variable, the "embedding_function" parameter set to the "embed_query" function of the "embeddings" object created earlier, and the "text_key" parameter set to "text".

In [26]:
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import Pinecone

embeddings = OpenAIEmbeddings(openai_api_key=OPENAI_API_KEY)

vectordb = Pinecone(
    index=index,
    embedding_function=embeddings.embed_query,
    text_key="text"
)

Initialize `gpt-3.5-turbo` chat model. 

In [27]:
from langchain.chat_models import ChatOpenAI

llm=ChatOpenAI(
    openai_api_key=OPENAI_API_KEY,
    temperature=0,
    model_name='gpt-3.5-turbo'
)

One additional thing we have here is the `chain_type="stuff"`. There are two options here, `"stuff"` or `"map_reduce"`. The `map_reduce` option essentially summarizes returned documents, whereas the `stuff` option just returns the retrieved documents as is.

The `retriever` is ready and can be used by us like this. However, we need to convert it into a `Tool` to be used by our conversational agent. To do that we need the `retriever` itself, a tool description, and a tool name. We use these to initialize the tool like so:

In [28]:
from langchain.chains import RetrievalQA

retriever = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=vectordb.as_retriever()
)

In [29]:
tool_desc = """Use this tool to answer user questions using Lex
Fridman podcasts. If the user states 'ask Lex' use this tool to get
the answer. This tool can also be used for follow up questions from
the user."""

This line of code imports the necessary class from the "langchain" library to create an agent.

This list represents the tools that the agent will use to answer questions. In this case, there is only one tool, which is a question-answering model that retrieves answers from a vector database.

The "Tool" object is initialized with several parameters.

The "func" parameter specifies the function that will be used to answer questions. In this case, the "run" method of the "retriever" object is used, which retrieves answers from the vector database.

The "description" parameter specifies a description of the tool. This parameter is set to the value of the "tool_desc" variable.

The "name" parameter specifies a name for the tool. In this case, the name is set to "Lex Fridman DB".

The "tools" list represents the tools that the agent will use to answer questions.

In [30]:
from langchain.agents import Tool

tools = [Tool(
    func=retriever.run,
    description=tool_desc,
    name='Lex Fridman DB'
)]

With that, we're ready to initialize the conversational agent. As it is a *conversational* agent, it does need some form of [conversational memory](https://www.pinecone.io/learn/langchain-conversational-memory/). For this we will use the `ConversationBufferWindowMemory` option, which will *remember* the previous `k` interactions between the user and the AI.

In [31]:
from langchain.chains.conversation.memory import ConversationBufferWindowMemory

memory = ConversationBufferWindowMemory(
    memory_key="chat_history",  # important to align with agent prompt (below)
    k=5,
    return_messages=True
)

Important items in `agent` parameter:

* `chat-conversational`: for chatbots with conversational memory.
* `react`: refers to the ReAct framework.
* `description`: because the LLM relies on the tool description to decide which tool to use.

In [32]:
from langchain.agents import initialize_agent

conversational_agent = initialize_agent(
    agent='chat-conversational-react-description', 
    tools=tools, 
    llm=llm,
    verbose=True,
    max_iterations=2,
    early_stopping_method="generate",
    memory=memory,
)

Conversational Agent Prompt
The prompt of the conversational agent is fairly complex. Let's create it then break it down.

In [33]:
conversational_agent.agent.llm_chain.prompt

ChatPromptTemplate(input_variables=['input', 'chat_history', 'agent_scratchpad'], output_parser=None, partial_variables={}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], output_parser=None, partial_variables={}, template='Assistant is a large language model trained by OpenAI.\n\nAssistant is designed to be able to assist with a wide range of tasks, from answering simple questions to providing in-depth explanations and discussions on a wide range of topics. As a language model, Assistant is able to generate human-like text based on the input it receives, allowing it to engage in natural-sounding conversations and provide responses that are coherent and relevant to the topic at hand.\n\nAssistant is constantly learning and improving, and its capabilities are constantly evolving. It is able to process and understand large amounts of text, and can use this knowledge to provide accurate and informative responses to a wide range of questions. Additionally, A

## Creating a System Message

This line of code creates a string variable "sys_msg" that contains a message that will be displayed to the user. This message is a system message that introduces the chatbot to the user.

The third line of code creates a prompt for the conversational agent with several parameters set.

This prompt represents the message that will be displayed to the user to prompt them to ask a question.

The "conversational_agent.agent.create_prompt()" function is called to create the prompt. The "system_message" parameter specifies the system message that will be displayed to the user. This parameter is set to the value of the "sys_msg" variable.

The "tools" parameter specifies the tools that the agent will use to answer questions. This parameter is set to the list of "Tool" objects that was created earlier.

The last line of code sets the prompt for the conversational agent. The "conversational_agent.agent.llm_chain" object represents the conversational agent, and the "prompt" variable represents the prompt that was created earlier.

When the agent is started, the prompt will be displayed to the user to prompt them to ask a question.


In [34]:
sys_msg = """You are a helpful chatbot that answers the user's questions.
"""

prompt = conversational_agent.agent.create_prompt(
    system_message=sys_msg,
    tools=tools
)
conversational_agent.agent.llm_chain.prompt = prompt

We can see the prompt template like so:

In [35]:
conversational_agent.agent.llm_chain.prompt

ChatPromptTemplate(input_variables=['input', 'chat_history', 'agent_scratchpad'], output_parser=None, partial_variables={}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], output_parser=None, partial_variables={}, template="You are a helpful chatbot that answers the user's questions.\n", template_format='f-string', validate_template=True), additional_kwargs={}), MessagesPlaceholder(variable_name='chat_history'), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], output_parser=None, partial_variables={}, template='TOOLS\n------\nAssistant can ask the user to use tools to look up information that may be helpful in answering the users original question. The tools the human can use are:\n\n> Lex Fridman DB: Use this tool to answer user questions using Lex\nFridman podcasts. If the user states \'ask Lex\' use this tool to get\nthe answer. This tool can also be used for follow up questions from\nthe user.\n\nRESPONSE FORMAT INSTRUCTION

The conversational agent prompt is defined by the `ChatPromptTemplate`. Let's break it down:

In [36]:
conversational_agent.agent.llm_chain.prompt.input_variables

['input', 'chat_history', 'agent_scratchpad']

 This prompt template contains *three* `input_variables`, those are:

* `input`: The new user input to the chatbot, i.e. our prompt/query.

* `chat_history`: We defined this above in the `ConversationBufferWindowMemory` definition.

* `agent_scratchpad`: This is where we store the thoughts of the LLM as it is deciding which tools to interact with and *how* to interact with them.

These `input_variables` are fed into the `messages` contained within the prompt template, let's see what we have there:

In [37]:
conversational_agent.agent.llm_chain.prompt.messages

[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], output_parser=None, partial_variables={}, template="You are a helpful chatbot that answers the user's questions.\n", template_format='f-string', validate_template=True), additional_kwargs={}),
 MessagesPlaceholder(variable_name='chat_history'),
 HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], output_parser=None, partial_variables={}, template='TOOLS\n------\nAssistant can ask the user to use tools to look up information that may be helpful in answering the users original question. The tools the human can use are:\n\n> Lex Fridman DB: Use this tool to answer user questions using Lex\nFridman podcasts. If the user states \'ask Lex\' use this tool to get\nthe answer. This tool can also be used for follow up questions from\nthe user.\n\nRESPONSE FORMAT INSTRUCTIONS\n----------------------------\n\nWhen responding to me please, please output a response in one of two formats:\n\n**Option 1:**\n


It's a little hard to see here, but there are **three** components in `messages`. Those are:

* `SystemMessagePromptTemplate`

* `MessagesPlaceholder`

* `HumanMessagePromptTemplate`

Let's start with the first item, the `SystemMessage`:

In [38]:
conversational_agent.agent.llm_chain.prompt.messages[0]

SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], output_parser=None, partial_variables={}, template="You are a helpful chatbot that answers the user's questions.\n", template_format='f-string', validate_template=True), additional_kwargs={})

In [39]:
print(
    conversational_agent.agent.llm_chain.prompt.messages[0].prompt.template
)

You are a helpful chatbot that answers the user's questions.



That is our initial system message that we set earlier with the `sys_msg`. There's not much to say about this other than it is used to "prime" (set the initial objective of) the model.

Next we have the `MessagesPlaceholder`:

In [40]:
conversational_agent.agent.llm_chain.prompt.messages[1]

MessagesPlaceholder(variable_name='chat_history')

We can see from `'chat_history'` (this must align to the `memory_key` from the `ConversationBufferWindowMemory` initialized earlier) that this is where the previous messages of the conversation will be fed into the LLM.

The format of this input is set by the type of conversational memory being used, which in this case is the `ConversationBufferWindowMemory`.

Finally, we have the `HumanMessagePromptTemplate`:

In [41]:
conversational_agent.agent.llm_chain.prompt.messages[2]

HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], output_parser=None, partial_variables={}, template='TOOLS\n------\nAssistant can ask the user to use tools to look up information that may be helpful in answering the users original question. The tools the human can use are:\n\n> Lex Fridman DB: Use this tool to answer user questions using Lex\nFridman podcasts. If the user states \'ask Lex\' use this tool to get\nthe answer. This tool can also be used for follow up questions from\nthe user.\n\nRESPONSE FORMAT INSTRUCTIONS\n----------------------------\n\nWhen responding to me please, please output a response in one of two formats:\n\n**Option 1:**\nUse this if you want the human to use a tool.\nMarkdown code snippet formatted in the following schema:\n\n```json\n{{\n    "action": string \\ The action to take. Must be one of Lex Fridman DB\n    "action_input": string \\ The input to the action\n}}\n```\n\n**Option #2:**\nUse this if you want to respond directly

In [42]:
print(
    conversational_agent.agent.llm_chain.prompt.messages[2].prompt.template
)

TOOLS
------
Assistant can ask the user to use tools to look up information that may be helpful in answering the users original question. The tools the human can use are:

> Lex Fridman DB: Use this tool to answer user questions using Lex
Fridman podcasts. If the user states 'ask Lex' use this tool to get
the answer. This tool can also be used for follow up questions from
the user.

RESPONSE FORMAT INSTRUCTIONS
----------------------------

When responding to me please, please output a response in one of two formats:

**Option 1:**
Use this if you want the human to use a tool.
Markdown code snippet formatted in the following schema:

```json
{{
    "action": string \ The action to take. Must be one of Lex Fridman DB
    "action_input": string \ The input to the action
}}
```

**Option #2:**
Use this if you want to respond directly to the human. Markdown code snippet formatted in the following schema:

```json
{{
    "action": "Final Answer",
    "action_input": string \ You should put 

This is the most interesting component. First, we have a single `input` — the user's query/prompt. But before this we see a lot of text, the majority of this is the setup for the LLM to be able to use any tools that we've passed to the conversational agent.

In our case, there is just one tool, the `Lex Fridman DB` tool that we defined earlier. We can also see the tool description that we defined. The LLM will use this tool description to figure out which tool (if any) it should use.

## Having a Conversation

Let's begin our conversation. We'll start as any typical conversation begins:

In [43]:
conversational_agent("What's up Lex?")



> Entering new AgentExecutor chain...
{
    "action": "Final Answer",
    "action_input": "I'm just a chatbot, so I don't have feelings, but I'm here to help you with any questions you have!"
}

> Finished chain.


{'input': 'hi how are you',
 'chat_history': [],
 'output': "I'm just a chatbot, so I don't have feelings, but I'm here to help you with any questions you have!"}

Looks good. We should note that there is this **AgentExecutor chain** thing. Where we can see an `"action"` and an `"action_input"`. It is here where the agent is deciding whether it should use a tool.

Here we see the agent decides on `"action": "Final Answer"`, meaning no tool is required. Therefore, it just uses the LLM as per usual to generate an answer. That answer can be seen in `"I'm just a chatbot, I don't have feelings, but thanks for asking! How can I assist you today?"`.

What if we mention the words `"ask lex"`?

In [45]:
conversational_agent("Are you scared of the future of AI?")



> Entering new AgentExecutor chain...
Here's your response:

```json
{
    "action": "Lex Fridman DB",
    "action_input": "What did Lex Fridman say about the future of AI?"
}
```
Observation: Lex Fridman has expressed concerns about the potential risks of AI and its impact on human civilization. He believes that the algorithms that drive our interaction on social media already have an intelligence and power that far outstrip the intelligence and power of any one human being. He also believes that the future of human civilization may be at stake over the question of the role of artificial intelligence in our society.
Thought:Here's your response:

```json
{
    "action": "Final Answer",
    "action_input": "Based on the insights from a certain AI expert, there are concerns about the potential risks of AI and its impact on human civilization. The expert believes that the future of human civilization may be at stake over the question of the role of artificial intelligence in our societ

{'input': 'ask lex about the future of ai',
 'chat_history': [HumanMessage(content='hi how are you', additional_kwargs={}),
  AIMessage(content="I'm just a chatbot, so I don't have feelings, but I'm here to help you with any questions you have!", additional_kwargs={})],
 'output': 'Based on the insights from a certain AI expert, there are concerns about the potential risks of AI and its impact on human civilization. The expert believes that the future of human civilization may be at stake over the question of the role of artificial intelligence in our society.'}

Great, we can see that the first thing the agent did was default to the `"Lex Fridman DB"` tool. The input to that tool was generated by the LLM, and is `"What did Lex Fridman say about the future of AI?"`.

This input is then passed into the `Lex Fridman DB` tool and the output observation of the LLM (after it has read all of the information returned by our vector DB is returned to our agent. From this observation the agent moves on to the `"Final Answer"` action, giving us the output.